In [12]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Read the Excel sheet
df = pd.read_excel('/content/drive/MyDrive/Important Records/Tishika Project/APT-Sheet.xlsx')

# Create a directed graph
G = nx.DiGraph()

# Add geographical location nodes
geographical_locations = df['Geographical_Location'].unique()
for location in geographical_locations:
    G.add_node(location, type='location')

# Add group nodes and connect them to their geographical locations
for index, row in df.iterrows():
    group_node = row['Group']
    location_node = row['Geographical_Location']
    G.add_node(group_node, **row.to_dict(), type='group')
    G.add_edge(location_node, group_node)

# Position the nodes using the spring layout
pos = nx.spring_layout(G)

# Create edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=1, color='#888'),
    hoverinfo='none',
    mode='lines'
)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

# Create node trace
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers+text',
    textposition='top center',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Attributes',
            xanchor='left',
            titleside='right'
        ),
    )
)

for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    if G.nodes[node]['type'] == 'group':
        node_info = f"Group: {node}<br>Associated Names: {G.nodes[node]['Associated_Names']}<br>Aliases: {G.nodes[node]['Aliases']}"
    else:
        node_info = f"Location: {node}"
    node_trace['text'] += tuple([node_info])

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='<br>Network graph of APT Groups by Geographical Location',
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="Network visualization of APT Groups by Geographical Location",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002)],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False))
                )

fig.show()


